In [1]:
import anndata as ad
import pandas as pd
import numpy as np

import umap
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import calinski_harabasz_score

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

input_dir = "/Users/apple/Desktop/KB/data/feat_RECOMB/LCL_larry200_train_test_with_diff_alpha"

adata_train = ad.read_h5ad('/Users/apple/Desktop/KB/data/LarryData/train_test/Larry_train.h5ad')
adata_test = ad.read_h5ad('/Users/apple/Desktop/KB/data/LarryData/train_test/Larry_test.h5ad')

train_labels = adata_train.obs["clone_id"].to_numpy()
test_labels = adata_test.obs["clone_id"].to_numpy()

 
train_embeddings = np.load(input_dir+'/feat_1025_larry_train_test__final_bs100_sf01/Larry_LCL_train_embeddings_bs100_sf01.npy')
test_embeddings = np.load(input_dir+'/feat_1025_larry_train_test__final_bs100_sf01/Larry_LCL_test_embeddings_bs100_sf01.npy')

print(train_labels.shape, test_labels.shape)
print(train_embeddings.shape, test_embeddings.shape)

(37207,) (3886,)
(37207, 64) (3886, 64)


#### calinski score

In [3]:
# Calculate the Calinski-Harabasz score
score = calinski_harabasz_score(train_embeddings, train_labels)

# Print the score
print("Train Calinski-Harabasz Score:", score)

Train Calinski-Harabasz Score: 90.36645200070247


## KNN classifier

In [4]:
adata_train.obs["clone_id"].value_counts()

clone_id
1261    159
2370    148
5714    126
292     120
5209    116
       ... 
1935      5
2216      5
5467      5
4404      5
4329      5
Name: count, Length: 2813, dtype: int64

### Test Accuracy

In [5]:

# Initialize the KNN classifier (you can adjust the number of neighbors)
knn = KNeighborsClassifier(n_neighbors=30)

# Train the KNN classifier
knn.fit(train_embeddings, train_labels)

# Predict the labels for the test set
y_pred = knn.predict(test_embeddings)

# Calculate the accuracy
accuracy = accuracy_score(test_labels, y_pred)

print(f"KNN classifier testing accuracy: {accuracy * 100:.2f}%")


/opt/anaconda3/envs/scvi-env/lib/python3.9/site-packages/threadpoolctl.py:1010: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


KNN classifier testing accuracy: 3.04%


### Train Accuracy

In [6]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(train_embeddings, train_labels, test_size=0.2, random_state=42)

# Initialize the KNN classifier (you can adjust the number of neighbors)
knn = KNeighborsClassifier(n_neighbors=30)

# Train the KNN classifier
knn.fit(X_train, y_train)

# Predict the labels for the test set
y_pred = knn.predict(X_test)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"KNN classifier training accuracy: {accuracy * 100:.2f}%")


KNN classifier training accuracy: 36.78%
